# 2. Data Understanding
The series Breaking Bad is a critically-acclaimed series with 62 episodes. We want to do a sentiment analysis of how the dialogues of different characters in Breaking Bad and see which characters speak the most, which words they use the most, and how their language changes over time.

We would require the series transcripts, to access the dialogues held in the show for this analysis. There are a number of sites that give acces to this.